## Investigating smoothed calibration solutions

* SNR = std$\left[\frac{1}{\frac{\mathrm{unsmoothed}}{\mathrm{smoothed}} - 1}\right]$

In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from astropy.time import Time
from pyuvdata import UVCal
import hera_cal
import hera_pspec as hp

In [ ]:
def freq_to_chan(freqs):
    freq_per_chan = 100. / 1024.
    chans = [(freq-100.) / freq_per_chan for freq in freqs]
    return chans

In [ ]:
JD_dec = '49088'
base_name = 'zen.2458106.' + JD_dec + '.xx.HH.uv'
abscal_file = '/data6/HERA/data/IDR2.1/2458106/' + base_name + '.abs.calfits'
smoothcal_file = '/data6/HERA/data/IDR2.1/2458106/' + base_name + '.smooth_abs.calfits'
im_freq_prefix = '/home/lwhitler/data/images/cal/noise/freq/' + base_name.replace('.uv', '')
im_time_prefix = '/home/lwhitler/data/images/cal/noise/time/' + base_name.replace('.uv', '')

In [ ]:
uvc_abs, uvc_smooth = UVCal(), UVCal()
uvc_abs.read_calfits(abscal_file)
uvc_smooth.read_calfits(smoothcal_file)

In [ ]:
ants = uvc_smooth.ant_array
flags = uvc_smooth.flag_array
cal_diff = (uvc_abs.gain_array/uvc_smooth.gain_array) - 1
cal_diff = np.ma.masked_array(cal_diff, flags)

#### Slice in frequency

Notes to self:
* Look at the skew and kurtosis -- there should be a `scipy` function for this, but needs to be for masked arrays to be safe

In [ ]:
plt.ioff()
for ant in ants[0:1]:
    print('Now doing antenna {0}.'.format(ant))
    ant_idx = np.where(uvc_abs.ant_array == ant)[0][0]
    for i in range(10):
        freqs = [100+10*i, 110+10*i]
        spw = np.floor(freq_to_chan(freqs)).astype(int)
        noise = cal_diff[ant_idx, 0, spw[0]:spw[1], :, 0]
        N = noise.count()
        
        if N > 0:
            real = np.real(noise.flatten())
            imag = np.imag(noise.flatten())
            mean_r, mean_i = real.mean(), imag.mean()
            quantiles_r, fit_r = stats.probplot(real, dist='norm')
            quantiles_i, fit_i = stats.probplot(imag, dist='norm')
            xfit_r = np.linspace(np.min(quantiles_r[0])-0.5, np.max(quantiles_r[0])+0.5)
            xfit_i = np.linspace(np.min(quantiles_i[0])-0.5, np.max(quantiles_i[0])+0.5)
            
            # Plot the data and the mean
            fig, ax = plt.subplots(1, 1, figsize=(8, 6))
            ax.axvline(0, color='k', ls=':', lw=1.25)
            ax.axvline(mean_r, color='#D11149', lw=1.25)
            ax.axvline(mean_i, color='#0A2463', lw=1.25)
            ax.hist(real, bins=100, alpha=0.4, color='#D11149', label='Real')
            ax.hist(imag, bins=100, alpha=0.4, color='#0A2463', label='Imaginary')
            
            # Plot the QQ plot
            ax_qq_r = fig.add_axes([0.675, 0.7, 0.25, 0.2])
            ax_qq_i = fig.add_axes([0.675, 0.5, 0.25, 0.2], sharex=ax_qq_r)
            ax_qq_r.plot(xfit_r, np.poly1d([fit_r[0], fit_r[1]])(xfit_r), color='k', lw=0.75)
            ax_qq_r.scatter(quantiles_r[0], quantiles_r[1], marker='.', s=0.6, color='#D11149', alpha=0.5)
            ax_qq_i.plot(xfit_i, np.poly1d([fit_i[0], fit_i[1]])(xfit_i), color='k', lw=0.75)
            ax_qq_i.scatter(quantiles_i[0], quantiles_i[1], marker='.', s=0.6, color='#0A2463')
            ax_qq_r.set_xlim([-4.15, 4.15])
            ax_qq_i.set_xlim([-4.15, 4.15])
            ax.text(0.804, 0.4, 'Theoretical Quantiles',
                    horizontalalignment='center', transform=ax.transAxes)
            ax.text(0.95, 0.72, 'Sample Quantiles', rotation=-90,
                    verticalalignment='center', transform=ax.transAxes)
            
            if i == 0:
                xlim = [-1.7, 1.7]
                xticks = np.arange(-1.5, 1.6, 0.3)
            else:
                xlim = [-0.65, 0.65]
                xticks = np.arange(-0.6, 0.7, 0.1)
            ax.set_xlim(xlim)
            ax.set_xticks(xticks)
            ax.set_xlabel(r'$\frac{\mathrm{abscal}}{\mathrm{smoothcal}} - 1$', fontsize=12)
            ax.set_title('Antenna {0}, {1}-{2} MHz'.format(ant, str(freqs[0]), str(freqs[1])))
            legend = ax.legend(loc='upper left', fontsize=10)
            legend.set_title('N={0}'.format(N), prop={'size':10})
            plt.tight_layout()
            plt.savefig(im_freq_prefix + '.ant' + str(ant) + '.' + str(freqs[0]) + '-' +
                        str(freqs[1]) + '.png', format='png')
            plt.close()
        else:
            print('No unflagged data for antenna {0} from {1}-{2} MHz, skipping...'.format(ant, str(freqs[0]), str(freqs[1])))

#### Slice in time

In [ ]:
plt.ioff()
time_arr_parts = uvc_abs.time_array.reshape(-1, 6)
for ant in ants:
    print('Now doing antenna {0}.'.format(ant))
    ant_idx = np.where(uvc_abs.ant_array == ant)[0][0]
    for time_arr in time_arr_parts:
        t1 = Time(time_arr[0], format='jd')
        t2 = Time(time_arr[-1], format='jd')
        date = t1.iso.split(' ')[0]
        t1_str = t1.iso.split(' ')[1]
        t2_str = t2.iso.split(' ')[1]

        t1_idx = np.where(uvc_abs.time_array == time_arr[0])[0][0]
        t2_idx = np.where(uvc_abs.time_array == time_arr[-1])[0][0]
        noise = cal_diff[ant_idx, 0, :, t1_idx:t2_idx, 0]
        N = noise.count()
        
        if N > 0:
            real = np.real(noise.flatten())
            imag = np.imag(noise.flatten())
            mean_r, mean_i = real.mean(), imag.mean()
            quantiles_r, fit_r = stats.probplot(real, dist='norm')
            quantiles_i, fit_i = stats.probplot(imag, dist='norm')
            xfit_r = np.linspace(np.min(quantiles_r[0])-0.5, np.max(quantiles_r[0])+0.5)
            xfit_i = np.linspace(np.min(quantiles_i[0])-0.5, np.max(quantiles_i[0])+0.5)
            
            # Plot the data and the mean
            fig, ax = plt.subplots(1, 1, figsize=(8, 6))
            ax.axvline(0, color='k', ls=':', lw=1.25)
            ax.axvline(mean_r, color='#D11149', lw=1.25)
            ax.axvline(mean_i, color='#0A2463', lw=1.25)
            ax.hist(real, bins=100, alpha=0.4, color='#D11149', label='Real')
            ax.hist(imag, bins=100, alpha=0.4, color='#0A2463', label='Imaginary')
            
            # Plot the QQ plot
            ax_qq_r = fig.add_axes([0.675, 0.7, 0.25, 0.2])
            ax_qq_i = fig.add_axes([0.675, 0.5, 0.25, 0.2], sharex=ax_qq_r)
            ax_qq_r.plot(xfit_r, np.poly1d([fit_r[0], fit_r[1]])(xfit_r), color='k', lw=0.75)
            ax_qq_r.scatter(quantiles_r[0], quantiles_r[1], marker='.', s=0.6, color='#D11149', alpha=0.5)
            ax_qq_i.plot(xfit_i, np.poly1d([fit_i[0], fit_i[1]])(xfit_i), color='k', lw=0.75)
            ax_qq_i.scatter(quantiles_i[0], quantiles_i[1], marker='.', s=0.6, color='#0A2463')
            ax_qq_r.set_xlim([-4.15, 4.15])
            ax_qq_i.set_xlim([-4.15, 4.15])
            ax.text(0.804, 0.4, 'Theoretical Quantiles',
                    horizontalalignment='center', transform=ax.transAxes)
            ax.text(0.95, 0.72, 'Sample Quantiles', rotation=-90,
                    verticalalignment='center', transform=ax.transAxes)
            
            ax.set_xlim([-1.1, 1.1])
            ax.set_xticks(np.arange(-1, 1.1, 0.2))
            ax.set_xlabel(r'$\frac{\mathrm{abscal}}{\mathrm{smoothcal}} - 1$', fontsize=12)
            ax.set_title('Antenna {0}, {1}, {2} to {3}'.format(ant, date, t1_str, t2_str))
            legend = ax.legend(loc='upper left', fontsize=10)
            legend.set_title('N={0}'.format(N), prop={'size':10})
            plt.tight_layout()
            plt.savefig(im_time_prefix.replace(JD_dec, '{0:.5f}'.format(t1.jd % 1).replace('0.','')) +
                        '.ant' + str(ant) + '.png', format='png')
            plt.close()
        else:
            print('No unflagged data for antenna {0} from {1}-{2}, skipping...'.format(ant, t1_str, t2_str))